In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import re  

# Load the Excel file
datasets_xlsx = pd.read_excel("TenDatasets.xlsx")

# Define all the helper functions again
def clean_columns (df):
    # Extract the ID from the column name
    df['ID'] = df['Original Column'].str.extract(r'(\d+)\.')
    
    # Split the column name from the description
    df[['Column', 'Descriptions']] = df['Original Column'].str.extract(r'\d+\.+\s*(.*?)(?:\s*\(|\s*\:|$)(.*)')
    
    # Split 'Descriptions' into multiple parts if there are multiple ":" characters
    descriptions = df['Descriptions'].str.split(':', expand=True)
    
    # Assign the multiple description fields
    for i in range(descriptions.shape[1]):
        df[f'Description {i+1}'] = descriptions[i].str.strip(' ()')
      
    # Trim leading/trailing white space
    df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
    
    # Replace empty strings with None
    df = df.replace(r'^\s*$', None, regex=True)
    
    # Drop the temporary 'Descriptions' column
    df = df.drop(columns=['Descriptions'])
    
    return df

def preprocess_columns(df):
    # Convert "Column" to lowercase to create "Standardized Column Name"
    df['Standardized Column Name'] = df['Column'].str.lower()
    
    # Remove '-' and '_' characters 
    df['Standardized Column Name'] = df['Standardized Column Name'].str.replace('[-_]', ' ', regex=True)
    
    return df

# Load the Excel file containing column names
columns_xlsx = pd.read_excel("AllColumnsFromTenDatasets.xlsx")

# Clean the columns
cleaned_columns_df  = clean_columns (columns_xlsx.copy())

# Preprocess the column names
preprocessed_columns_df = preprocess_columns(cleaned_columns_df )

dictionary = {}

# Open the file and read line by line
with open("dictionary.txt", "r") as file:
    for line in file:
        # Remove the trailing newline and comma, then split the line into key and value at the colon
        key, value = line.rstrip(",\n").split(":")

        # Remove the quotes around the key and value
        key = key.strip("'")
        value = value.strip("'")

        # Add the key-value pair to the dictionary
        dictionary[key] = value

abbreviations_dict = {}
with open("abbreviations_dictionary.txt", "r") as file:
    for line in file:
        abbr, full_form = line.strip().split(":")
        abbreviations_dict[abbr.strip()] = full_form.strip()

def replace_abbreviations(text, abbreviations_dict):
    words = text.split()
    replaced_words = [abbreviations_dict.get(word, word) for word in words]
    return " ".join(replaced_words)

# Extract unique target words and their corresponding analysis results
#target_words_analysis = analysis_xlsx[['target_word_found', 'analysis of Standardized Column Name']].dropna().drop_duplicates()
#description_words_analysis = analysis_xlsx[['description_word_found', 'analysis of description_found']].dropna().drop_duplicates()

target_words_dict = dictionary

# Create dictionaries for mapping
#target_words_dict = dict(zip(target_words_analysis['target_word_found'], target_words_analysis['analysis of Standardized Column Name']))
#description_words_dict = dict(zip(description_words_analysis['description_word_found'], description_words_analysis['analysis of description_found']))
description_words_dict = dictionary

def apply_analysis(df, target_words_dict, description_words_dict, abbreviations_dict):
    # Initialize new columns
    df['target_word_found'] = None
    df['analysis of Standardized Column Name'] = None
    df['description_word_found'] = None
    df['analysis of description_found'] = None

    # Apply target words analysis
    for i, row in df.iterrows():
        # Skip if 'Standardized Column Name' is missing
        if pd.isnull(row['Standardized Column Name']):
            continue

        found = False
        for word, analysis in target_words_dict.items():
            if re.search(rf'\b{word}\b', row['Standardized Column Name'], re.IGNORECASE):
                df.at[i, 'target_word_found'] = word
                df.at[i, 'analysis of Standardized Column Name'] = analysis
                found = True
                break

        # If no match found, replace abbreviations and try again
        if not found:
            replaced_text = replace_abbreviations(row['Standardized Column Name'], abbreviations_dict)
            for word, analysis in target_words_dict.items():
                if re.search(rf'\b{word}\b', replaced_text, re.IGNORECASE):
                    df.at[i, 'target_word_found'] = word
                    df.at[i, 'analysis of Standardized Column Name'] = analysis
                    found = True
                    break

        # If still no match, break down the word into substrings
        if not found:
            for j in range(len(row['Standardized Column Name']), 2, -1):
                for k in range(len(row['Standardized Column Name']) - j + 1):
                    subword = row['Standardized Column Name'][k:k+j]
                    
                    # Check if the subword exists in the abbreviation dictionary
                    expanded_subword = abbreviations_dict.get(subword, None)
                    if expanded_subword:
                        # If the expanded subword exists in the target words dictionary, use it
                        if expanded_subword in target_words_dict:
                            df.at[i, 'target_word_found'] = expanded_subword
                            df.at[i, 'analysis of Standardized Column Name'] = target_words_dict[expanded_subword]
                            found = True
                            break
                        
                    # Else, continue with the original subword
                    elif subword in target_words_dict:
                        df.at[i, 'target_word_found'] = subword
                        df.at[i, 'analysis of Standardized Column Name'] = target_words_dict[subword]
                        break

                if found:
                    break

    # Apply description words analysis
    for i, row in df.iterrows():
        # Skip if 'Description 1' and 'Description 2' are missing
        if pd.isnull(row['Description 1']) and pd.isnull(row['Description 2']):
            continue

        for word, analysis in description_words_dict.items():
            if ((not pd.isnull(row['Description 1']) and re.search(rf'\b{word}\b', row['Description 1'], re.IGNORECASE)) or 
               (not pd.isnull(row['Description 2']) and re.search(rf'\b{word}\b', row['Description 2'], re.IGNORECASE))):
                df.at[i, 'description_word_found'] = word
                df.at[i, 'analysis of description_found'] = analysis
                break

    return df

# Apply this function to the dataframe
analysed_columns_df = apply_analysis(preprocessed_columns_df.copy(), target_words_dict, description_words_dict, abbreviations_dict)

def analysis_of_column(std_col_analysis, desc_found_analysis):
    # Handle the case where only one column has a value
    if pd.isnull(std_col_analysis):
        return desc_found_analysis
    elif pd.isnull(desc_found_analysis):
        return std_col_analysis

    # Handle the exceptions
    if (std_col_analysis.startswith("numerical") and desc_found_analysis.startswith("numerical")) or \
        std_col_analysis in ["telephone format", "numerical >= 0 and PK", "numerical or string"]:
        return std_col_analysis

    # If no exceptions apply, return the value from "analysis of description_found"
    return desc_found_analysis

# Apply the function to create the new column
analysed_columns_df['Analysis of Column'] = analysed_columns_df.apply(lambda row: analysis_of_column(row['analysis of Standardized Column Name'], row['analysis of description_found']), axis=1)

# Save the results to a new Excel file
analysed_columns_df.to_excel("AnalysedColumns.xlsx", index=False)